In [ ]:
//# Dyrektywy preprocesora zawierają bibliotekę MPI i biblioteki standardowe do wejścia i wyjścia, a także czas.

%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
#include <time.h>
 
// moja funkcja
//# Funkcja funkcja oblicza kwadrat swojego argumentu.
double funkcja(double x)
{
  return x*x;
}

//# Funkcja główna inicjuje MPI, określa rangę i liczbę procesów, a następnie oblicza wielkość kroku h numerycznej integracji.
int main(int argc, char **argv)
{
    int numer_procesu;
    int liczba_procesow;
    int tag=1;
    
    MPI_Status status;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &numer_procesu);
    MPI_Comm_size(MPI_COMM_WORLD, &liczba_procesow);
    clock_t t;
    double time_taken;

    double l_procesow = liczba_procesow, n_procesu = numer_procesu;

    double granica_dolna_calki = 1,granica_gorna_calki = 4;
    double h = (granica_gorna_calki - granica_dolna_calki) / l_procesow;
    
    //# Jeśli ranga bieżącego procesu wynosi 0 (główny proces), otrzymuje on wartości całkowanej z innych procesów, dodaje je i wyświetla wynik.
    if(numer_procesu == 0)
    {
      double calka = 0 ,x=0;
      for(int i=1; i<liczba_procesow; i++)
      {
        MPI_Recv(&x, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        calka +=x;
      }
      printf("Wartosc funkcji(dolnej granicy calki)/2 * h = %f \n",funkcja(granica_dolna_calki)/2 * h);
      printf("Wartosc funkcji(gornej granicy calki)/2 * h = %f \n",funkcja(granica_gorna_calki)/2 * h);

      printf("Wartosc calki = %1f \n", h * (funkcja(granica_dolna_calki)/2 + calka 
        + funkcja(granica_gorna_calki)/2));
    }

    //# Jeśli ranga bieżącego procesu nie wynosi 0, oblicza on wartość całkowanej na swoim przydzielonym przedziale, wysyła wynik do głównego procesu i wyświetla czas wykonania.
    if(numer_procesu != 0)
    {
        t = clock();
        double xi,wynik;
        n_procesu = numer_procesu;
        xi = granica_dolna_calki + (n_procesu/l_procesow) 
            * (granica_gorna_calki - granica_dolna_calki);
        wynik = funkcja(xi);
        printf("Wynik procesu %d = %f \t", numer_procesu,wynik*h);
        MPI_Send(&wynik, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
        t = clock() - t;
        time_taken = ((double)t)/CLOCKS_PER_SEC;
        printf("Czas wykonywania sie procesu nr %d = %f\n",numer_procesu,time_taken);
    }

    MPI_Finalize();
}
//# Biblioteka MPI jest zakończona na końcu.
EOF
mpicc pi-mpi.c && mpirun -n 1 --allow-run-as-root a.out